# BioNeMo - Geneformer inferencing for single cell downstream tasks

This tutorial showcases how to run the BioNeMo container, pre-train a geneformer model, and use it for inferencing downstream single cell tasks. At the end of this tutorial, a user will learn:
- launching the BioNeMo container
- Download data from czi to use for pre-training and inference.
- Convert AnnData files into the sparse CSR memmap format used by BioNeMo
- Kick-off pretraining with a custom single cell dataset
- Restore the pre-trained model and perform inference with the same czi dataset.


### Prerequisites:
- BioNeMo Framework container is running (refer to the [Getting Started](../index.md) section)
- Familiarity with some components of the BioNeMo framework such as the [Models](../models/megamolbart.md) and [Inferencing](../inference-grpc-fw.md)


#### Running the BioNeMo container

This example has been built by launching the container in a local machine with 2 x A6000 RTX GPUs. Refer to specific instructions for [remote and multi-node launch]

Once the container is launched, navigate to http://0.0.0.0:8888, http://localhost:8888, or the IP address of the workstation/node. A JupyterLab instance should show up.

#### Copy this code and input files into JupyterLab

In the launched JupyterLab, run the codes in a Jupyter notebook as provided in the code cells below.

## Getting example single cell data and setting it up for inference

First, we must acquire single cell training data for inference. To do this we will install the cellxgene-census api and download a small dataset. We use the example provided by the czi api examples page to download a single h5ad file. Generally, our workflow expects a collection of h5ad files to be used for pre-training. In this case, we restrict to 100k cells from a single dataset  to keep training time and downloading time small.

In [ ]:
!pip install cellxgene-census

In [2]:
# Below are paths required for setting up pre-training and inference.
tutorial_data_dir = "/workspace/bionemo/data/singlecell_tutorial/download_anndata"
train_tutorial_data_dir = "/workspace/bionemo/data/singlecell_tutorial/download_anndata/train"
val_tutorial_data_dir = "/workspace/bionemo/data/singlecell_tutorial/download_anndata/val"
test_tutorial_data_dir = "/workspace/bionemo/data/singlecell_tutorial/download_anndata/test"

train_tutorial_processed_dir = "/workspace/bionemo/data/singlecell_tutorial/processed_data/train"
val_tutorial_processed_dir = "/workspace/bionemo/data/singlecell_tutorial/processed_data/val"
test_tutorial_processed_dir = "/workspace/bionemo/data/singlecell_tutorial/processed_data/test"
tutorial_output_dir = "/workspace/bionemo/data/singlecell_tutorial/inference_output"
tutorial_output_inference_pickle = f"{tutorial_output_dir}/human_covid19_bcells_from_scratch.pkl"
demo_data_train_download_path = f"{train_tutorial_data_dir}/human_covid19_bcells.h5ad"
demo_data_val_download_path = f"{val_tutorial_data_dir}/human_covid19_bcells.h5ad"
demo_data_test_download_path = f"{test_tutorial_data_dir}/human_covid19_bcells.h5ad"

In [3]:
!mkdir -p {train_tutorial_data_dir}
!mkdir -p {val_tutorial_data_dir}
!mkdir -p {test_tutorial_data_dir}
!mkdir -p {train_tutorial_processed_dir}
!mkdir -p {val_tutorial_processed_dir}
!mkdir -p {test_tutorial_processed_dir}
!mkdir -p {tutorial_output_dir}

In [4]:
import cellxgene_census
frac_train = 0.8
frac_val = 0.1
frac_test = 0.1

with cellxgene_census.open_soma(census_version="2023-12-15") as census:
    filter1 = "cell_type == 'B cell' and tissue_general == 'lung' and disease == 'COVID-19' and is_primary_data == True"

    adata = cellxgene_census.get_anndata(
        census = census,
        organism = "Homo sapiens",
        obs_value_filter = filter1,
    )
    n_train = int(adata.shape[0] * frac_train)
    n_val = int(adata.shape[0] * frac_val)
    n_test = adata.shape[0] - n_train - n_val
    # Create some splits, bad practice since ordering may be a thing but let's just take ranges for this demo.
    adata_train = adata[0:n_train].copy()
    adata_val = adata[n_train:(n_train+n_val)].copy()
    adata_test = adata[(n_train+n_val):].copy()
    adata_train.write(demo_data_train_download_path)
    adata_val.write(demo_data_val_download_path)
    adata_test.write(demo_data_test_download_path)

In [5]:
!ls -laht {demo_data_train_download_path}
!ls -laht {demo_data_val_download_path}
!ls -laht {demo_data_test_download_path}

-rw-r--r-- 1 gkaushik domain-users 22M Sep 12 10:03 /workspace/bionemo/data/singlecell_tutorial/download_anndata/train/human_covid19_bcells.h5ad
-rw-r--r-- 1 gkaushik domain-users 12M Sep 12 10:03 /workspace/bionemo/data/singlecell_tutorial/download_anndata/val/human_covid19_bcells.h5ad
-rw-r--r-- 1 gkaushik domain-users 13M Sep 12 10:03 /workspace/bionemo/data/singlecell_tutorial/download_anndata/test/human_covid19_bcells.h5ad


In [ ]:
# Create training data processed directory
!python /workspace/bionemo/bionemo/data/singlecell/sc_memmap.py \
  --data-path {train_tutorial_data_dir} \
  --save-path {train_tutorial_processed_dir}

# Create validation data processed directory
!python /workspace/bionemo/bionemo/data/singlecell/sc_memmap.py \
  --data-path {val_tutorial_data_dir} \
  --save-path {val_tutorial_processed_dir}

# Create test data processed directory
!python /workspace/bionemo/bionemo/data/singlecell/sc_memmap.py \
  --data-path {test_tutorial_data_dir} \
  --save-path {test_tutorial_processed_dir}

In [7]:
!ls -laht {train_tutorial_processed_dir}
!ls -laht {test_tutorial_processed_dir}
!ls -laht {val_tutorial_processed_dir}

total 13M
-rw-r--r-- 1 gkaushik domain-users 157K Sep 12 10:05 features.csv
drwxr-xr-x 2 gkaushik domain-users 4.0K Sep 12 10:05 .
-rw-r--r-- 1 gkaushik domain-users 5.9M Sep 12 10:05 gene_expression_ind.npy
-rw-r--r-- 1 gkaushik domain-users  15K Sep 12 10:05 gene_expression_ptr.npy
-rw-r--r-- 1 gkaushik domain-users 5.9M Sep 12 10:05 gene_expression_data.npy
-rw-r--r-- 1 gkaushik domain-users 1.1M Sep 12 10:05 metadata.json
drwxr-xr-x 5 gkaushik domain-users 4.0K Sep 12 10:00 ..
total 4.5M
-rw-r--r-- 1 gkaushik domain-users  19K Sep 12 10:06 features.csv
drwxr-xr-x 2 gkaushik domain-users 4.0K Sep 12 10:06 .
-rw-r--r-- 1 gkaushik domain-users 1.7M Sep 12 10:06 gene_expression_ind.npy
-rw-r--r-- 1 gkaushik domain-users 1.9K Sep 12 10:06 gene_expression_ptr.npy
-rw-r--r-- 1 gkaushik domain-users 1.7M Sep 12 10:06 gene_expression_data.npy
-rw-r--r-- 1 gkaushik domain-users 1.1M Sep 12 10:06 metadata.json
drwxr-xr-x 5 gkaushik domain-users 4.0K Sep 12 10:00 ..
total 3.2M
-rw-r--r-- 1 gka

# Pretraining
Now that we have aquired the h5ad files we would like to use for training and converted them to a sparse memmap. We will kickoff training. This involves two distinct steps
- preprocessing (indicated with do_training=False), where artifacts are downloaded from huggingface to be used by the model. Importantly, we set the `dataset_path` to be the same place we created the sparse memmap files. This is how BioNeMo knows where to find files for training, including both training data and additional artifacts (such as tokenizers).
- pretraining, where the model is actually trained.

We set the flag `max_steps` to limit the runtime. Check the full config file in `examples/singlecell/geneformer/conf` for a complete list of arguments.

In [ ]:
# Run preprocessing to acquire the requisite files for pre-training.
!python /workspace/bionemo/examples/singlecell/geneformer/pretrain.py \
  ++model.data.train_dataset_path={train_tutorial_processed_dir} \
  ++model.data.val_dataset_path={val_tutorial_processed_dir} \
  ++model.data.test_dataset_path={test_tutorial_processed_dir} \
  ++do_training=False

In [ ]:
# Pretrain the model using
!python /workspace/bionemo/examples/singlecell/geneformer/pretrain.py \
  --config-dir /workspace/bionemo/examples/singlecell/geneformer/conf \
  --config-name geneformer_config \
  ++model.data.train_dataset_path={train_tutorial_processed_dir} \
  ++model.data.val_dataset_path={val_tutorial_processed_dir} \
  ++model.data.test_dataset_path={test_tutorial_processed_dir} \
  ++trainer.devices=1 \
  ++trainer.max_steps=200 \
  ++exp_manager.exp_dir={tutorial_output_dir} \
  ++exp_manager.wandb_logger_kwargs.project="geneformer_pretrain_test" \
  ++model.data.output_fname={tutorial_output_inference_pickle} \
  ++exp_manager.wandb_logger_kwargs.offline=True \
  ++exp_manager.resume_if_exists=False \
  ++do_training=True

# Running inference.

We can see from the above training job that the model was trained for a short number of steps. Note the end of the log file the experiment manager leaves a message about where the resulting `.nemo` file is written. This file is used for finetuning, inference, or training from an existing set of model weights. See the example produced below from our run:

```text
[NeMo I 2024-04-26 22:02:36 nemo_model_checkpoint:183] New .nemo model saved to: /workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer/2024-05-13_16-33-16/checkpoints/geneformer.nemo
[NeMo I 2024-04-26 22:02:36 nlp_overrides:412] Removing checkpoint: /workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer/2024-05-13_16-33-16/checkpoints/geneformer--val_loss=8.70-step=100-consumed_samples=800.0-last.ckpt
Epoch 0: 100%|█| 200/200 [00:27<00:00,  7.17it/s, v_num=2-05, reduced_train_loss`Trainer.fit` stopped: `max_steps=200` reached.
Epoch 0: 100%|█| 200/200 [00:27<00:00,  7.17it/s, v_num=2-05, reduced_train_loss
```

We will take the `.nemo` file logged:
`/workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer/2024-04-26_22-02-05/checkpoints/geneformer.nemo`

and use this for inference.

In [16]:
# Find latest checkpoint file

import os
from datetime import datetime

directory = '/workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer'
timestamped_dirs = []

for dir_name in os.listdir(directory):
    dir_path = os.path.join(directory, dir_name)
    if os.path.isdir(dir_path):
        try:
            timestamp = datetime.strptime(dir_name, "%Y-%m-%d_%H-%M-%S")
            timestamped_dirs.append((dir_name, timestamp))
        except ValueError:
            pass

latest_dir = max(timestamped_dirs, key=lambda x: x[1])
pretrained_nemo_file = f'/workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer/{latest_dir[0]}/checkpoints/geneformer.nemo'
print(pretrained_nemo_file)

/workspace/bionemo/data/singlecell_tutorial/inference_output/geneformer/2024-09-12_10-07-12/checkpoints/geneformer.nemo


In [ ]:
# Run inference on test
!python /workspace/bionemo/bionemo/model/infer.py \
  --config-dir /workspace/bionemo/examples/singlecell/geneformer/conf \
  --config-name infer \
  ++model.downstream_task.restore_from_path={pretrained_nemo_file} \
  ++model.data.dataset_path={test_tutorial_processed_dir} \
  ++exp_manager.exp_dir={tutorial_output_dir} \
  ++model.data.output_fname={tutorial_output_inference_pickle} 

# Load inference result and cluster with UMAP.
Now we will inspect our result. First, we expect there to be one prediction for each cell, we can compare the shape of the anndata object to the predictions produced by our model. After this, we can simply pass our embeddings into umap, and view the result! In this case its a very poorly trained model with very few cells, so keep expectations low!

The inference_results pickle file contains one set of hiddens and embeddings for each cell. The hiddens contain the embedding per-token, whereas the embeddings contain the mean embedding for all gene tokens with special tokens (CLS, MASK, etc) removed.

In [18]:
import pickle
with open(tutorial_output_inference_pickle, 'rb') as inference_handle:
    inference_results = pickle.load(inference_handle)
len(inference_results), adata.shape, inference_results[0].keys()
#NBVAL_CHECK_OUTPUT

(232, (2313, 60664), dict_keys(['embeddings']))

In [19]:
inference_results[0]['embeddings'].shape
#NBVAL_CHECK_OUTPUT

(256,)

In [20]:
import umap
reducer = umap.UMAP()
embedding = reducer.fit_transform([x['embeddings'] for x in inference_results])

In [21]:
embedding.shape
#NBVAL_CHECK_OUTPUT

(232, 2)

In [22]:
assert adata_test.obs.shape[0] == len(inference_results)

In [ ]:
from matplotlib import pyplot as plt

results = adata_test.obs.copy()
results['x'] = embedding[:, 0]
results['y'] = embedding[:, 1]

covariates = ["assay", "development_stage", "dataset_id", "sex"]
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, figsize=(10,10))

for ax,covar in zip(axes.flat, covariates):
    for cov, cov_df in results.groupby(covar):
        ax.scatter(
            cov_df.x,
            cov_df.y,
            s=3,
            alpha=0.75,
            label=cov,
        )
    if len(results[covar].unique()) < 8:
        ax.legend()
    ax.set_title(f"Embeddings by {covar}")

In [ ]:
adata_test.obs.columns